In [40]:
import re
import csv
import json
import logging
import traceback
import numpy as np
import pandas as pd

import orjson

#import dask.dataframe as dd
#from dask.distributed import Client

In [2]:
logging.basicConfig(
    filename='../logs/app.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger('data_analysis')

### Load

In [3]:
path = '../data/order_table_202208161552.csv'
df = pd.read_csv(path, sep=';',
    parse_dates=['created_at_irpf', 'created_at_loan'],
    dtype={'bank_code_pl': str,
            'branch_number_pl': str,
            'loan_id': str})

In [49]:
with open("../data/order_table_202208161552.csv") as csvfile:  
    data = csv.DictReader(csvfile)
    for row in data:
        print(row['loan_id'])

KeyError: 'loan_id'

ValueError: I/O operation on closed file.

In [4]:
path = '../data/Dicionario_Grafia_Banco_SRF-v14.sav'
bank_df = pd.read_spss(path)
bank_df = bank_df.rename(columns={
    'BankName': 'bank',
    'Codigo_Banco': 'bank_code'})

In [5]:
path = '../data/bank_branch.parquet'
branch_df = pd.read_parquet(path)
branch_df = branch_df.rename(columns={
    'Bank': 'bank_code',
    'Branch': 'branch'})

In [6]:
df.head()

,person_id,loan_id,irpf_id,created_at_irpf,created_at_loan,safra_created,product_code,state,bank_code_pl,branch_number_pl,rev,value,ordem
0,14936917,12794324,32377897,2022-03-04,2022-03-04,202203,PERSONAL,5.0,33,3055,10,"{""rev"":10,""objType"":""IrpfPersonInfoT"",""personI...",1
1,13661860,12833025,32928842,2022-04-15,2022-04-15,202204,PERSONAL_GERU,3.0,260,1,10,"{""rev"":10,""objType"":""IrpfPersonInfoT"",""personI...",1
2,978784,12825333,32905275,2022-04-06,2022-04-06,202204,PERSONAL,3.0,1,NaN,6,"{""rev"":6,""objType"":""IrpfPersonInfoT"",""personId...",1
3,15106591,NaN,32885418,2022-03-28,NaT,202203,NaN,NaN,NaN,NaN,3,"{""rev"":3,""objType"":""IrpfPersonInfoT"",""personId...",1
4,4146904,12825443,32905717,2022-04-06,2022-04-06,202204,PERSONAL,3.0,341,NaN,10,"{""rev"":10,""objType"":""IrpfPersonInfoT"",""personI...",1


### Transformation

In [7]:
### general json and dict functions

def get_json_value(df: pd.DataFrame, col: str):
    '''Takes a pandas dataframe and a string column-name.
    Extracts json object from specified column in dataframe.
    Returns original dataframe joined with normalized json as columns.'''

    try:
        df = df.copy()
    except Exception as e:
        logger.debug(str(e))
        raise(e)

    try:
        data = pd.json_normalize(
            df[col].apply(
                orjson.loads), max_level=0)
    except KeyError as e:
        logger.debug(str(e))
        return df
    else:
        col_lst = data.columns.difference(df.columns)
        return df.join(data[col_lst])

def extract_value_dict(data: dict, key: str, default=np.nan):
    '''Function receives dictionary with key string
    and returns value. If default is provided, returns
    default value when key does not exist, otherwise returns nan.'''

    try:
        status = data.get(key, default)
    except AttributeError as e:
        logger.debug(str(e))
        raise(e)

    return status

def map_normalize_dict(df: pd.DataFrame, col: str, map:dict):
    '''Receives Pandas dataframe, column name and
    dictionary containing new column names as keys and dict
    keys as values. Normalizes dict column in dataframe and returns
    original dataframe with new columns.'''

    df = df.copy()

    for new_col_name, dict_key in map.items():
        df.loc[:, new_col_name] = df[col].apply(
            lambda x: extract_value_dict(
                data=x, key=dict_key))
                
    return df.fillna(value=np.nan)

In [8]:
### text functions

def apply_regex_series(series: pd.Series, regex: re.Pattern, handle_nan=True):
    '''Receives Pandas Series and regex and returns a numpy array containing
    1 for every match and 0 for no match. Use handle_nan parameter if you want
    to return 0 when value is nan, otherwise nan is passed to regex.'''

    if handle_nan:
        return (np.where(
            series.str.contains(regex) & series.notna(), 1, 0))
    else:
        return (np.where(
            series.str.contains(regex), 1, 0))

In [9]:
### irpf functions

def explode_dict_col(df: pd.DataFrame, dict_col='riskInfo'):
    '''Explodes risk_dict where each row is a
    tax report year.'''

    df = df.copy()

    df.loc[:, 'tax_report_data'] = (
        df[dict_col].apply(
            lambda x: x.values()))

    df = df.apply(pd.Series.explode).reset_index(drop=True).copy()

    return df

def get_irpf_status(df: pd.DataFrame, text_col: str):
    '''Receives pandas dataframe and column name and applies
    regex to column to generate new columns representing status
    of irpf application. Returns dataframe with new columns.'''

    df = df.copy()

    regex_not_consulted = re.compile(
        r'(?:^\s*$|\bdata\sde\snascimento\sinformada\b'
        r'.*\bestá\sdive|\bnão\scoletado'
        r'|\bocorreu\suma\sinconsistência\s?[.])'
        , re.IGNORECASE)

    regex_not_declared = re.compile(
        r'(?:\bconsta\sapresentação\sde\sdeclaração\sanual'
        r'\sde\sisento\b|\bapresentação\sda\sdeclaração\s'
        r'como\sisento\b|\bdeclaração\sconsta\scomo\sisento\b'
        r'|\bdeclaração\sconsta\scomo\spedido\sde'
        r'\sregularização\b|\bsua\sdeclaração\snão\sconsta'
        r'\sna\sbase\sde\sdados\b|\bainda\snão\sestá\sna'
        r'\sbase\b)', re.IGNORECASE)

    regex_tax_refund = re.compile(
        r'(?:\bsituação\sda\srestituição[:]\screditada\b'
        r'|\bsomente\sserá\spermitida\spor\smeio\sdo\scódigo\sde\sacesso\b'
        r'|\baguardando\sreagendamento\spelo\scontribuinte[.]?'
        r'|\bdevolvida\sà\sreceita\sfederal[,]?\sem\srazão\sdo\snão\sresgate\b'
        r'|\benviada\spara\scrédito\sno\sbanco\b'
        r'|\breagendada\spara\scrédito\sno\sbanco\b'
        r'|\bdados\sda\sliberação\sde\ssua\srestituição\b'
        r'|\bdeclaração\sestá\sna\sbase\sde\sdados\b'
        r'|\bestá\sna\sbase[,]\sutilize\so\sextrato\b'
        r'|\bdeclaração\sjá\sfoi\sprocessada[.]?$'
        r'|\brestituição[:]\saguardando\sdevolução\spelo\sbanco\b)'
        , re.IGNORECASE)
    
    col_list = ['irpf_extraction_error', 'irpf_not_declared', 'irpf_tax_refund']

    df.loc[:, 'irpf_extraction_error'] = apply_regex_series(
        df[text_col], regex_not_consulted, handle_nan=False)
    df.loc[:, 'irpf_not_declared'] = apply_regex_series(
        df[text_col], regex_not_declared)
    df.loc[:, 'irpf_tax_refund'] = apply_regex_series(
        df[text_col], regex_tax_refund)

    df.loc[:, 'irpf_tax_to_pay'] = df[col_list].apply(
        lambda x: 1 not in x.values, axis=1).astype(int)

    return df

def retrieve_stars(y: int, x: int, star_arr: np.array):
    try:
        if y >= 16:
            stars = 5
        else:
            stars = star_arr[y][x]
    except IndexError:
        return -1
    else:
        return stars

def set_star_number(arr_declarations: np.array, arr_refunds: np.array):
    base_arr = [
        [0],
        [1, 1],
        [1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 2, 2],
        [1, 1, 2, 2, 3, 3],
        [1, 2, 2, 3, 3, 4, 4],
        [2, 2, 3, 3, 4, 4, 4, 5],
        [2, 3, 3, 4, 4, 4, 5, 5, 5],
        [2, 3, 4, 4, 4, 5, 5, 5, 5, 5],
        [3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5],
        [3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [3, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
        [4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]

    max_len = np.array([len(arr) for arr in base_arr]).max()

    default_value = -1

    star_array = [
        np.pad(arr, (0, max_len - len(arr)),
        mode='constant',
        constant_values=default_value) for arr in base_arr]

    #ret_func = np.vectorize(retrieve_stars)
    #return ret_func(arr_declarations, arr_refunds, star_array)
    return np.array([retrieve_stars(y, x, star_array)
                        for y, x in zip(arr_declarations, arr_refunds)])

In [10]:
cols = ['person_id', 'loan_id', 'irpf_id',
        'created_at_irpf', 'product_code',
        'state', 'rev', 'riskInfo', 'bank_code_pl', 'branch_number_pl']

col_key_map = {
    'cpf': 'cpf',
    'full_status_text': 'full_status_text',
    'bank': 'bank',
    'branch': 'branch'}

bank_df = bank_df.fillna('###')

df['bank_code_pl'] = df['bank_code_pl'].str.zfill(3)
df['branch_number_pl'] = df['bank_code_pl'].str[:4].str.zfill(4)

df = (
    df.pipe(get_json_value, 'value')[cols]
    .pipe(explode_dict_col)
    .pipe(map_normalize_dict, 'tax_report_data', col_key_map)
    .pipe(get_irpf_status, 'full_status_text')
    ).rename(columns={'riskInfo': 'year'})

df = df.merge(bank_df, on='bank', how='left')
df = df.merge(branch_df[['bank_code', 'branch', 'branch_code']],
                on=['bank_code', 'branch'], how='left')
df = df.merge(
    branch_df[['bank_code',
                'branch',
                'branch_code']].rename(
                    columns={'bank_code': 'bank_code_pl',
                            'branch': 'branch_number_pl',
                            'branch_code': 'branch_code_pl'}),
    on=['bank_code_pl', 'branch_number_pl'], how='left')

In [11]:
gp_estr = df.groupby(['cpf', 'created_at_irpf']).agg(
                number_declaration=('tax_report_data', 'count'),
                number_tax_refund=('irpf_tax_refund', 'sum')
                ).reset_index()
        
gp_estr['ESTR'] = set_star_number(gp_estr.number_declaration.values, gp_estr.number_tax_refund.values)

In [34]:
branch_codes = ['PERS', 'STIL', 'PRIM', 'OUTR', 'HSBC', 'VANG', 'UNIC', 'ESPA', 'PRIV']

dtypes = {
    'branch_code': pd.CategoricalDtype(categories=branch_codes)
}

gp_branch = pd.get_dummies(df.astype(dtypes),
                columns=['branch_code'],
                prefix='',
                prefix_sep='').groupby(
                    ['cpf', 'created_at_irpf']
                    )[branch_codes].sum().reset_index()

In [38]:
gp_estr.merge(
    gp_branch,
    on=['cpf', 'created_at_irpf'], how='left').head(20)

,cpf,created_at_irpf,number_declaration,number_tax_refund,ESTR,PERS,STIL,PRIM,OUTR,HSBC,VANG,UNIC,ESPA,PRIV
0,10000632830,2022-03-25,6,1,2,0,0,0,0,0,0,0,0,1
1,10000898430,2022-04-18,10,0,3,0,0,0,0,0,0,0,0,0
2,10001036416,2022-05-27,10,0,3,0,0,0,0,0,0,0,0,0
3,10001529609,2022-04-05,4,2,1,0,0,0,0,0,0,0,2,0
4,10001529609,2022-04-07,10,7,5,0,0,0,0,0,0,0,5,0
5,10002062623,2022-05-12,10,0,3,0,0,0,0,0,0,0,0,0
6,10003733637,2022-03-22,10,0,3,0,0,0,0,0,0,0,0,0
7,10005124794,2022-03-21,9,0,2,0,0,0,0,0,0,0,0,0
8,10005384435,2022-05-11,10,0,3,0,0,0,0,0,0,0,0,0
9,10005566622,2022-03-16,10,0,3,0,0,0,0,0,0,0,0,0


In [18]:
def get_presumed_income():
    estr_dic = {
        0: 1300,
        1: 2500,
        3: 4000
    }

,person_id,loan_id,irpf_id,created_at_irpf,product_code,state,rev,year,bank_code_pl,branch_number_pl,...,branch_code_pl,ESPA,HSBC,OUTR,PERS,PRIM,PRIV,STIL,UNIC,VANG
0,14936917,12794324,32377897,2022-03-04,PERSONAL,5.0,10,2012,033,0033,...,VANG,0,0,0,0,0,0,0,0,0
1,14936917,12794324,32377897,2022-03-04,PERSONAL,5.0,10,2013,033,0033,...,VANG,0,0,0,0,0,0,0,0,0
2,14936917,12794324,32377897,2022-03-04,PERSONAL,5.0,10,2014,033,0033,...,VANG,0,0,0,0,0,0,0,0,0
3,14936917,12794324,32377897,2022-03-04,PERSONAL,5.0,10,2015,033,0033,...,VANG,0,0,0,0,0,0,0,0,0
4,14936917,12794324,32377897,2022-03-04,PERSONAL,5.0,10,2016,033,0033,...,VANG,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165812,15002564,NaN,32914626,2022-04-09,NaN,NaN,10,2017,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
165813,15002564,NaN,32914626,2022-04-09,NaN,NaN,10,2018,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
165814,15002564,NaN,32914626,2022-04-09,NaN,NaN,10,2019,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
165815,15002564,NaN,32914626,2022-04-09,NaN,NaN,10,2020,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0


In [14]:
# aux = get_irpf_status(df, 'full_status_text').copy()
# aux.shape